In [ ]:
#Author: Anh Phan (GitHub: @anhphan1, Email: anhphan@udel.edu, Phone: (302) 345-5885)

%pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
%pip install keras
%pip install tensorflow
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
# The code here takes the ticker symbol from any publicly traded company and downloads the stock price data from Yahoo Finance.
stock = 'AAPL'
data = yf.download(stock, start='2020-01-01', end='2022-01-01')

In [3]:
data['Daily_Return'] = data['Close'].pct_change().fillna(0)
data['Volatility'] = data['Daily_Return'].rolling(window=60).std().fillna(0)

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Historical Volatility of ' + stock)
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.plot(data['Volatility'])
plt.show()

In [ ]:
scaled_volatility = scaler.fit_transform(data['Volatility'].values.reshape(-1, 1))

In [ ]:
train_volatility = scaled_volatility[0:int(len(scaled_volatility)*0.95)]
x_train_vol, y_train_vol = [], []
for i in range(60, len(train_volatility)):
    x_train_vol.append(train_volatility[i-60:i, 0])
    y_train_vol.append(train_volatility[i, 0])
x_train_vol, y_train_vol = np.array(x_train_vol), np.array(y_train_vol)
x_train_vol = np.reshape(x_train_vol, (x_train_vol.shape[0], x_train_vol.shape[1], 1))


In [ ]:
model_volatility = Sequential()
model_volatility.add(LSTM(units=50, return_sequences=True, input_shape=(x_train_vol.shape[1], 1)))
model_volatility.add(LSTM(units=50))
model_volatility.add(Dense(1))
model_volatility.compile(optimizer='adam', loss='mean_squared_error')
model_volatility.fit(x_train_vol, y_train_vol, epochs=10, batch_size=64)

In [ ]:
test_volatility = scaled_volatility[int(len(scaled_volatility)*0.95)-60:]
x_test_vol, y_test_vol = [], data['Volatility'].values[int(len(scaled_volatility)*0.95):]
for i in range(60, len(test_volatility)):
    x_test_vol.append(test_volatility[i-60:i, 0])
x_test_vol = np.array(x_test_vol)
x_test_vol = np.reshape(x_test_vol, (x_test_vol.shape[0], x_test_vol.shape[1], 1))

In [ ]:
predictions_volatility = model_volatility.predict(x_test_vol)
predictions_volatility = scaler.inverse_transform(np.reshape(predictions_volatility, (predictions_volatility.shape[0], 1)))

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Prediction vs Real Volatility of ' + stock)
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.plot(y_test_vol, label='True Volatility')
plt.plot(predictions_volatility, label='Predicted Volatility')
plt.legend()
plt.show()

In [ ]:
rmse_volatility = np.sqrt(mean_squared_error(y_test_vol, predictions_volatility))
print("The Root Mean Squared Error (RMSE) for volatility is:", rmse_volatility)

In [ ]:
forecast_days = 30
input_seq = scaled_data[-60:].tolist()

for _ in range(forecast_days):
    # Use the last 60 days to predict the next day
    next_input = np.array(input_seq[-60:])
    next_input = np.reshape(next_input, (1, 60, 1))
    
    predicted_price = model.predict(next_input)
    
    # Append the predicted price to the input sequence
    input_seq.append(predicted_price[0].tolist())

# Extract the forecasted days and inverse transform for real price
forecasted_prices = scaler.inverse_transform(input_seq[-forecast_days:])

# Plot the results
plt.figure(figsize=(12,6))
plt.title(f'Forecasted Stock Price of {stock} for Next {forecast_days} Days')
plt.xlabel('Days')
plt.ylabel('Close Price (USD)')
plt.plot(forecasted_prices, label='Forecasted Price')
plt.legend()
plt.show()